# EverymanBio Fungi DNA Barcoding Chromatogram Explorer

Let's open an .ab1 file, aka trace file, aka chromatogram, using BioPython.

In [1]:
from Bio import SeqIO
chromatogram = SeqIO.read('trace_files/oc.laedaporous-ITS1-F.ab1', 'abi')

Let's take a look at some of the annotation data in the file.

In [70]:
import pandas as pd

annotations_df = pd.DataFrame.from_dict(chromatogram.annotations,
                                        orient='index',
                                        dtype='str',
                                        columns=['Value'])
display(annotations_df)

,Value
sample_well,D11
dye,Z-BigDyeV3
polymer,POP7
machine_model,3730
run_start,2020-11-25 20:47:19
run_finish,2020-11-25 22:03:24
abif_raw,"{'AEPt1': 9925, 'AEPt2': 9925, 'APFN2': b'KBan..."
molecule_type,DNA
